In [21]:
import pandas as pd
import numpy as np

In [22]:
df1 = pd.read_csv('2024_feb_unduplicated_orders.csv')
df1['title'] = df1['title'].str.lstrip()
df1['title'] = df1['title'].str.rstrip()
df1.shape

(1177, 22)

In [23]:
df1_notnull = df1[df1['customs_description'].notnull()]
df1_null = df1[df1['customs_description'].isna()]
df1_notnull.shape, df1_null.shape

((1029, 22), (148, 22))

In [24]:
56366 + 2592

58958

In [25]:
temp_df = pd.read_csv('product_feed.csv')
temp_df['PRODUCT TYPE'] = temp_df['PRODUCT TYPE'].str.replace('&gt;', '>')
temp_df['PRODUCT TYPE'] = temp_df['PRODUCT TYPE'].str.replace('&amp;', '&')
temp_df.rename(columns={'TITLE': 'title'}, inplace=True)
temp_stock_df = temp_df['title'].str.split('\(|\)', expand=True).iloc[:,[0,1]]
temp_df['title'] = temp_stock_df[0]
temp_df['title'] = temp_df['title'].str.lstrip()
temp_df['title'] = temp_df['title'].str.rstrip()
temp_df.drop_duplicates(subset=['title'], inplace=True)
temp_df = temp_df[['title', 'PRODUCT TYPE']]

<>:5: SyntaxWarning: invalid escape sequence '\('
<>:5: SyntaxWarning: invalid escape sequence '\('
C:\Users\retai\AppData\Local\Temp\ipykernel_12928\158616370.py:5: SyntaxWarning: invalid escape sequence '\('
  temp_stock_df = temp_df['title'].str.split('\(|\)', expand=True).iloc[:,[0,1]]


In [26]:
df1_null = df1_null.merge(temp_df, on='title', how='left')
df1_null['customs_description'] = df1_null['PRODUCT TYPE']
df1_null.drop(['PRODUCT TYPE'], axis=1, inplace=True)
df1_null.shape

(148, 22)

In [16]:
df1 = pd.concat([df1_notnull, df1_null])
df1.shape

(1177, 22)

In [17]:
df1['customs_description'].isnull().sum()

53

In [18]:
df2 = df1['customs_description'].str.split(' > ', expand=True)
df1.drop(['F_Cat', 'S_Cat', 'T_Cat'], axis=1, inplace=True)
df2.columns = ['F_Cat', 'S_Cat', 'T_Cat']
df1 = df1.join(df2)
df1['Category'] = df1['F_Cat'] + ' > ' + df1['S_Cat']

In [19]:
df1.to_csv('2024_feb_unduplicated_orders fixed_categories.csv')

In [20]:
df1[df1['title'] == 'Mens Leather Hodded Jacket Black: Super Bomber']

,order_id,order_state,order_date,grand_total_inc,title,line,model,reference,quantity,price_inc,...,customs_description,date,time,day,fifo_quantity,Category,colour,F_Cat,S_Cat,T_Cat
2,77921,Awaiting Dispatch,2024-02-29 21:51:05,199.000,Mens Leather Hodded Jacket Black: Super Bomber,92816,super bomber,4946506,1,199.00,...,Mens > Leather Jackets > Bomber Styles,2024-02-29,21:51:05,29,1,Mens > Leather Jackets,Black,Mens,Leather Jackets,Bomber Styles
2,77921,Awaiting Dispatch,2024-02-29 21:51:05,199.000,Mens Leather Hodded Jacket Black: Super Bomber,92816,super bomber,4946506,1,199.00,...,Mens > Leather Jackets > Bomber Styles,2024-02-29,21:51:05,29,1,Ladies > Ladies Waistcoats,Black,Ladies,Ladies Waistcoats,None
195,77744,Order Dispatched,2024-02-24 11:46:12,189.050,Mens Leather Hodded Jacket Black: Super Bomber,92619,super bomber,4946505,1,199.00,...,Mens > Leather Jackets > Bomber Styles,2024-02-24,11:46:12,24,1,Mens > Leather Jackets,Black,Mens,Leather Jackets,Bomber Styles
259,77687,Order Dispatched,2024-02-22 15:32:05,189.000,Mens Leather Hodded Jacket Black: Super Bomber,92548,super bomber,4946504,1,199.00,...,Mens > Leather Jackets > Bomber Styles,2024-02-22,15:32:05,22,1,Mens > Leather Jackets,Black,Mens,Leather Jackets,Bomber Styles
299,77655,Order Dispatched,2024-02-21 20:29:09,190.825,Mens Leather Hodded Jacket Black: Super Bomber,92505,super bomber,4946506,1,165.83,...,Mens > Leather Jackets > Bomber Styles,2024-02-21,20:29:09,21,1,Mens > Leather Jackets,Black,Mens,Leather Jackets,Bomber Styles
